## Basic Code

In [2]:
import findspark
findspark.init('/home/venkat/Downloads/spark-3.2.0-bin-hadoop3.2')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('cluster').getOrCreate()

In [3]:
from pyspark.ml.clustering import KMeans

In [5]:
data = spark.read.format('libsvm').load('sample_kmeans_data.txt')

21/11/30 09:28:38 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [8]:
data.show() #lables are not really useful here

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [9]:
final_data=data.select('features')

In [11]:
kmeans = KMeans().setK(2).setSeed(1)

In [12]:
kmean_model=kmeans.fit(final_data)

In [13]:
kmean_model

KMeansModel: uid=KMeans_d97a48990b8c, k=2, distanceMeasure=euclidean, numFeatures=3

In [17]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()
sse = evaluator.evaluate(kmean_model.transform(final_data))
print(sse)

0.9997530305375207


In [19]:
centers = kmean_model.clusterCenters()

In [20]:
centers # get the two centers

[array([9.1, 9.1, 9.1]), array([0.1, 0.1, 0.1])]

In [21]:
results = kmean_model.transform(final_data)

In [23]:
results.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         1|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+

